In [ ]:
!pip install sentence_transformers

In [ ]:
!pip install requirements.txt

In [ ]:
from elasticsearch import Elasticsearch

In [ ]:
es = Elasticsearch(
    "https://localhost:9200",
    basic_auth=("elastic","+z8wCDJ=3Z-nDAoZExt*"),
    ca_certs="/Users/ryeon/Documents/YerinShin/2024_1/http_ca.crt"
    #ca_certs="/Users/abidsaudagar/Personal/yt1_semantic_search/elasticsearch-8.13.0/config/certs/http_ca.crt"
)
es.ping()

In [ ]:
import pandas as pd

df = pd.read_csv("alltagging20_tfidf20_test.csv")

In [ ]:
df.head()

In [ ]:
for index, keyword in enumerate(keywords):
    print(keyword)

In [ ]:
df = df[['Column1', '상품이름', '가격', '상품정보','가장 비중이 높은 키워드']]

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/LaBSE')

In [ ]:
df["DescriptionVector"] = df["상품정보"].apply(lambda x: model.encode(x))

In [ ]:
import numpy as np

# '문장' 열을 상품별로 합치기
df_sentences_combined = df.groupby('상품이름')['문장'].apply(lambda x: ' '.join(x)).reset_index()

# 상품별 DescriptionVector 평균 내기
df_vector_combined = df.groupby('상품이름')['DescriptionVector'].apply(lambda x: np.mean(x, axis=0)).reset_index()

# 가격 열은 상품당 하나의 가격으로 가정하여 첫 번째 값을 사용
df_price = df.groupby('상품이름')['가격'].first().reset_index()

df_column = df.groupby('상품이름')['Column1'].first().reset_index()

df_topic = df.groupby('상품이름')['태깅num'].first().reset_index()

# 합치기
df_combined2 = df_sentences_combined.merge(df_vector_combined, on='상품이름', suffixes=('_sentences', '_vector'))
df_combined2 = df_combined2.merge(df_price, on='상품이름')
df_combined2 = df_combined2.merge(df_column, on='상품이름')
df_combined2 = df_combined2.merge(df_topic, on='상품이름')

# 열 이름 변경
df_combined2.columns = ['상품이름', '문장', 'DescriptionVector', '가격', 'Column1', '태깅num']

# 결과 확인
df_combined2

In [ ]:
# 열 순서 변경
df_new2 = df_combined2[['Column1', '상품이름', '가격', '문장', '태깅num', 'DescriptionVector']]

# 'Column1' 기준으로 데이터 나열
df_new2.sort_values(by='Column1', inplace=True)

# 결과 확인
df_new2

In [ ]:
indexMapping = {
    "properties":{  
        "ProductID":{
            "type" : "text"
        },
        "ProductName":{
            "type":"text"
        },
        "Price (INR)":{
            "type":"long"
        },
        "Description":{
            "type":"text"
        },
        "taggingNum":{
            "type":"long"
        },
        "DescriptionVector":{
            "type":"dense_vector",
            "dims": 768,
            "index":True,
            "similarity": "cosine"
        }

    }
}

In [ ]:
#from indexMapping import indexMapping

es.indices.create(index="tagging_with", mappings=indexMapping)

In [ ]:
record_list = df.to_dict("records")

In [ ]:
record_list

In [ ]:
for record in record_list:
    try:
        es.index(index="tagging_with", document=record)
    except Exception as e:
        print(e)

In [ ]:
es.count(index="tagging_with")